In [11]:
import numpy as np
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from fuzzywuzzy import fuzz
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from itertools import chain
from nltk.collocations import *
import nltk, re, string, collections
from nltk.util import ngrams 

C:\ProgramData\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\ProgramData\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\ProgramData\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\ProgramData\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [12]:
path="D:/scrapping/magicbricks/"
df=pd.read_csv(f'{path}data_gnr_feb.csv')

In [13]:
df.head()

,name,id,createdDate,description,Society,url,price,priceInWord,cityName,addressLocality,...,floorno,furnshingstatus,agentName,agentCompanyName,agentMaskedmobilenumber,projectName,builderName,propertyType,locality,postedBy
0,2 BHK Apartment for Sale in Golfforeste AC Apa...,domcache_srp_46630105,20200211,"2 Bath,Unfurnished,16 floor Paramount Golf For...",Golfforeste AC Apartments,https://www.magicbricks.com/propertyDetails/2-...,3710000.0,37.1 Lac,Greater Noida,Zeta 1,...,16,Unfurnished,Praveen Choudhary,BD Green Home Pvt. Ltd.,+91-88XXXXXXX90,Golfforeste AC Apartments,Paramount Group,Apartment,Zeta 1,Agent
1,4 BHK Apartment for Sale in Purvanchal Royal C...,domcache_srp_46527183,20200131,"4 Bath,Semi-Furnished,19 floor A life where yo...",Purvanchal Royal City,https://www.magicbricks.com/propertyDetails/4-...,11200000.0,1.12 Cr,Greater Noida,Chi 5,...,19,Semi-Furnished,Shashi Kapoor,Purvanchal Projects Pvt. Ltd.,+91-78XXXXXXX42,Purvanchal Royal City,Purvanchal Projects Pvt. Ltd.,Apartment,Chi 5,Builder
2,"3 BHK Apartment for Sale in SKA Metro Ville, E...",domcache_srp_46365097,20200130,"2 Bath,Semi-Furnished SKA Metro Ville is the f...",SKA Metro Ville,https://www.magicbricks.com/propertyDetails/3-...,3430000.0,34.3 Lac,Greater Noida,"Eta 2, Eta",...,NaN,Semi-Furnished,SKA GROUP,NaN,+91-92XXXXXXX15,SKA Metro Ville,SKA Group,Apartment,"Eta 2, Eta",Builder
3,"2 BHK Apartment for Sale in Arihant Ambar, Gre...",domcache_srp_34690341,20200207,"2 Bath,Semi-Furnished,20 floor Arihant Ambar i...",Arihant Ambar,https://www.magicbricks.com/propertyDetails/2-...,4300000.0,43 Lac,Greater Noida,Greater Noida West,...,20,Semi-Furnished,Rajat Verma,Arihant Group,+91-88XXXXXXX26,Arihant Ambar,Arihant Group,Apartment,Greater Noida West,Builder
4,"1 BHK Apartment for Sale in Prideville , Yamun...",domcache_srp_45300261,20200210,"1 Bath,Semi-Furnished We at Prideville underst...",Prideville,https://www.magicbricks.com/propertyDetails/1-...,3700000.0,37 Lac,Greater Noida,Yamuna Expressway,...,NaN,Semi-Furnished,Vipin,Imperia Structures Ltd.,+91-74XXXXXXX93,Prideville,Imperia Structures Ltd.,Apartment,Yamuna Expressway,Agent


In [4]:
df.shape

(1533, 26)

In [5]:
def pre_process(text):
    
    # lowercase
    text=text.lower()
    
    #remove tags
    text=re.sub("</?.*?>"," <> ",text)
    
    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    
    return text

In [6]:
df['clean_desc'] = df['description'].apply(lambda x:pre_process(x))

In [7]:
df.clean_desc[0]

' bath unfurnished floor paramount golf forest ac apartments complimented by designer landscape with a green golf course for you and your family a well deserved healthy lifestyle '

In [9]:
def get_stop_words(stop_file_path):
    """load stop words """
    
    with open(stop_file_path, 'r', encoding="utf-8") as f:
        stopwords = f.readlines()
        stop_set = set(m.strip() for m in stopwords)
        return frozenset(stop_set)

In [10]:
stopwords=get_stop_words("stopwords.txt")

In [11]:
stopwords

frozenset({'#',
           '#  Freely available stopword list, balancing coverage and size.',
           '#  From http://www.lextek.com/manuals/onix/stopwords1.html',
           '#  stopwords.txt',
           'a',
           'about',
           'above',
           'across',
           'after',
           'again',
           'against',
           'all',
           'almost',
           'alone',
           'along',
           'already',
           'also',
           'although',
           'always',
           'among',
           'an',
           'and',
           'another',
           'any',
           'anybody',
           'anyone',
           'anything',
           'anywhere',
           'are',
           'area',
           'areas',
           'around',
           'as',
           'ask',
           'asked',
           'asking',
           'asks',
           'at',
           'away',
           'b',
           'back',
           'backed',
           'backing',
           'backs',
        

In [12]:
docs=df['clean_desc'].tolist()

In [13]:
cv=CountVectorizer(max_df=0.85,stop_words=stopwords)
word_count_vector=cv.fit_transform(docs)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['available', 'balancing', 'com', 'coverage', 'freely', 'html', 'http', 'lextek', 'list', 'manuals', 'onix', 'size', 'stopword', 'stopwords', 'stopwords1', 'txt', 'www'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [14]:
word_count_vector.shape

(1533, 2862)

In [16]:

cv=CountVectorizer(max_df=0.85,stop_words=stopwords,max_features=1000)
word_count_vector=cv.fit_transform(docs)
word_count_vector.shape

(1533, 1000)

In [17]:

list(cv.vocabulary_.keys())[:10]

['unfurnished',
 'paramount',
 'golf',
 'ac',
 'apartments',
 'designer',
 'landscape',
 'green',
 'course',
 'family']

In [19]:
list(cv.get_feature_names())[950:1000]

['verdant',
 'via',
 'vicinity',
 'view',
 'views',
 'vihar',
 'villa',
 'village',
 'villas',
 'ville',
 'visit',
 'visitor',
 'visitors',
 'vistas',
 'vitrified',
 'waiting',
 'walk',
 'walking',
 'wall',
 'wardrobe',
 'wardrobes',
 'wares',
 'washroom',
 'waste',
 'water',
 'weary',
 'welcoming',
 'west',
 'western',
 'whereas',
 'wi',
 'wide',
 'widely',
 'windows',
 'wise',
 'wonderful',
 'wood',
 'wooden',
 'woodwork',
 'world',
 'worth',
 'xu',
 'yamuna',
 'yatharth',
 'yoga',
 'yourself',
 'yrd',
 'yrs',
 'zeta',
 'zone']

In [21]:
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [22]:
tfidf_transformer.idf_

array([5.33990171, 6.25619244, 6.03304889, 5.93773871, 5.50242064,
       4.67207234, 6.94933962, 6.72619607, 6.25619244, 5.07753744,
       6.54387451, 6.72619607, 6.38972383, 5.391195  , 5.00342947,
       6.38972383, 6.72619607, 6.72619607, 6.38972383, 5.69657665,
       6.72619607, 4.08713874, 6.72619607, 6.38972383, 6.1384094 ,
       5.391195  , 4.2084996 , 6.72619607, 5.391195  , 5.391195  ,
       4.67207234, 6.25619244, 6.38972383, 6.38972383, 6.38972383,
       6.03304889, 7.23702169, 6.38972383, 6.72619607, 3.7923392 ,
       6.03304889, 6.72619607, 6.03304889, 4.50699259, 6.94933962,
       1.39347728, 4.72471607, 6.38972383, 5.62758378, 5.56304526,
       5.56304526, 6.54387451, 5.69657665, 6.38972383, 6.54387451,
       7.23702169, 6.38972383, 6.72619607, 5.56304526, 6.72619607,
       5.56304526, 5.56304526, 4.75211504, 6.1384094 , 6.94933962,
       5.24459153, 6.94933962, 6.1384094 , 6.54387451, 6.72619607,
       1.68924347, 6.25619244, 6.03304889, 4.55144435, 5.15758

In [27]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
url_symbol=re.compile('[^https?:\/\/.*[\r\n]*]')
STOPWORDS = set(stopwords.words('english'))

In [30]:
def clean_text(text):
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text =url_symbol.sub('',text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text

In [31]:
df['clean_desc'] = df['description'].apply(lambda x:clean_text(x))

In [32]:
df['clean_desc'][0]

'2 bath unfurnished 16 floor paramount golf forest ac apartments complimented designer landscape green golf course family welldeserved healthy lifestyle'

In [33]:
df['sent_tokenize']=df['clean_desc'].apply(lambda x:sent_tokenize(x))

In [35]:
df['sent_tokenize'][1]

['4 bath semifurnished 19 floor life live abundant exhilarating moments youve always craved stunning contemporary design redefines classic urban environment luxury amenities distinctive lifestyle enlivened welcoming mother nature complete grand living experience immerse enchanting luxury enjoy best life store purvanchal royal city']

In [36]:
df['word_tokenize']=df['clean_desc'].apply(lambda x:word_tokenize(x))

In [49]:
def get_top_n_trigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(1, 3)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

In [59]:
common_words = get_top_n_trigram(df['clean_desc'])

In [60]:
common_words

[('floor', 1968),
 ('bath', 1529),
 ('apartment', 1216),
 ('noida', 1025),
 ('multistorey', 1014),
 ('multistorey apartment', 1011),
 ('greater', 881),
 ('greater noida', 869),
 ('available', 830),
 ('semifurnished', 754),
 ('bath semifurnished', 718),
 ('sale', 713),
 ('available sale', 643),
 ('facing', 640),
 ('bhk', 636),
 ('unfurnished', 615),
 ('apartment available', 586),
 ('bath unfurnished', 584),
 ('apartment available sale', 584),
 ('multistorey apartment available', 535),
 ('lac', 532),
 ('bhk multistorey', 485),
 ('bhk multistorey apartment', 485),
 ('east', 473),
 ('floor bhk', 457),
 ('east facing', 435),
 ('located', 423),
 ('floor bhk multistorey', 422),
 ('floors', 392),
 ('semifurnished floor', 376),
 ('bath semifurnished floor', 375),
 ('total', 373),
 ('unfurnished floor', 321),
 ('bath unfurnished floor', 318),
 ('flat', 299),
 ('north', 278),
 ('apartment located', 278),
 ('floor multistorey', 267),
 ('floor multistorey apartment', 266),
 ('building', 244),
 ('mu

In [61]:
df5 = pd.DataFrame(common_words, columns = ['description' , 'count'])

In [71]:
df5=df5.head(100)

In [72]:
df5

,description,count
0,floor,1968
1,bath,1529
2,apartment,1216
3,noida,1025
4,multistorey,1014
...,...,...
95,living,102
96,residents,102
97,balconies,101
98,facilities,98


In [75]:
keywords=df5['description'].unique()

In [76]:
keywords

array(['floor', 'bath', 'apartment', 'noida', 'multistorey',
       'multistorey apartment', 'greater', 'greater noida', 'available',
       'semifurnished', 'bath semifurnished', 'sale', 'available sale',
       'facing', 'bhk', 'unfurnished', 'apartment available',
       'bath unfurnished', 'apartment available sale',
       'multistorey apartment available', 'lac', 'bhk multistorey',
       'bhk multistorey apartment', 'east', 'floor bhk', 'east facing',
       'located', 'floor bhk multistorey', 'floors',
       'semifurnished floor', 'bath semifurnished floor', 'total',
       'unfurnished floor', 'bath unfurnished floor', 'flat', 'north',
       'apartment located', 'floor multistorey',
       'floor multistorey apartment', 'building',
       'multistorey apartment located', 'furnished', 'property', 'area',
       'west', 'floor north', 'building total', 'north east',
       'floor east', 'floor east facing', 'bath furnished',
       'north east facing', 'room', 'tower', 'covere

In [77]:
features="I want 2 BHK with study apartment with 1400 sqft between 6 to 12 floor in north-east facing in Greater Noida West"

In [78]:
features=clean_text(features)

In [79]:
features

'want 2 bhk study apartment 1400 sqft 6 12 floor northeast facing greater noida west'

In [80]:
common_test=get_top_n_trigram(features)

ValueError: Iterable over raw text documents expected, string object received.

In [ ]:
list3 = set(list1)&set(list2) # we don't need to list3 to actually be a list

list4 = sorted(list3, key = lambda k : list1.index(k))

In [83]:
import spacy
import en_core_web_sm

In [84]:
nlp=en_core_web_sm.load()

In [93]:
df.Society.unique()

array(['Golfforeste AC Apartments', 'Purvanchal Royal City',
       'SKA Metro Ville', 'Arihant Ambar', 'Prideville ', 'Gaur City',
       'ATS Dolce', 'Page Three Residences', 'Stellar One',
       'Rise Resort Residences', 'Godrej Golf Links', 'Eros Sampoornam I',
       'Gaur Saundaryam', 'SKA Greenarch', 'Sikka Kaamya Greens',
       'Nirala Greenshire', 'Sublimis', 'Gaur Yamuna City',
       'Meridian Towers', 'Paramount Golf Foreste', 'Migsun Vilaasa', nan,
       'Mahagun Mantra 2', 'Migsun Ultimo', 'ATS Green Paradiso',
       'Himalaya Pride', 'Vrinda City', 'Nimbus Express Park View II',
       'Supertech Golf Village', 'Mahagun Mantra 1', 'Gaur Atulyam',
       'Nimbus Express Park View I', 'Czar Suites', 'Ajnara Le Garden',
       'Supertech Up Country', 'Ajnara Panorama', 'Patel Neotown',
       'Purvanchal Heights', 'Assotech Springfields', 'Ajnara Homes',
       'Supertech Upcountry', 'AVJ Heights', 'NRI Residency',
       'Victoryone Amara', 'Eros Sampoornam', 'Technoci

In [129]:
text=nlp(df['description'][173])

In [106]:
df[df.Society=="ATS Destinaire"]['description']

141    3 Bath,Semi-Furnished,10 floor,North - East fa...
173    4 Bath,Semi-Furnished,5 floor,North facing Spe...
Name: description, dtype: object

In [107]:
text

4 Bath,Semi-Furnished,5 floor,North facing Special inaugural offer AC  Modular KitchenFor SOFT LAUNCH BOOKINGS.ONLY project offering all the towers with 2 apartments per floor at Greater Noida W.ATS Destinaire Greater Noida is an ultimate reflection of the urban chic lifestyle located in Sector1, Greater Noida. The project hosts in its lap exclusively designed Residential Apartments, each being an epitome of elegance and simplicity. In the busy streets of one of city’s most premium areas, it’s not easy to find a lifestyle that has it all. And yet, here is one unique DESTINATION with everything you’ve always dreamt of! Impressiv

In [108]:
for word in text.ents:
    print(word.text,word.label_)

4 CARDINAL
Semi-Furnished,5 ORG
AC ORG
2 CARDINAL
Greater Noida ORG
Destinaire Greater Noida PERSON
Sector1 GPE
Greater Noida PERSON
Residential Apartments ORG
one CARDINAL


In [113]:
spacy.explain('GPE')

'Countries, cities, states'

In [111]:
from spacy import displacy

In [112]:
displacy.render(text,style='ent',jupyter=True)

In [122]:
import nltk
from nltk.corpus import stopwords

In [123]:
cachedStopWords = stopwords.words("english")

In [132]:
def testFuncNew(x):
        
        text = x
        text = re.sub(r'[^a-zA-Z0-9\s]', ' ', text)
        text = ' '.join([word for word in text.split() if word not in cachedStopWords])
        return text

In [133]:
type(text)

str

In [134]:
text=testFuncNew(df['description'][173])

In [135]:
text

'4 Bath Semi Furnished 5 floor North facing Special inaugural offer AC Modular KitchenFor SOFT LAUNCH BOOKINGS ONLY project offering towers 2 apartments per floor Greater Noida W ATS Destinaire Greater Noida ultimate reflection urban chic lifestyle located Sector1 Greater Noida The project hosts lap exclusively designed Residential Apartments epitome elegance simplicity In busy streets one city premium areas easy find lifestyle And yet one unique DESTINATION everything always dreamt Impressiv'

In [136]:
text=nlp(text)

In [137]:
for word in text.ents:
    print(word.text,word.label_)

4 CARDINAL
Bath Semi ORG
5 CARDINAL
North PRODUCT
AC Modular ORG
LAUNCH BOOKINGS PERSON
2 CARDINAL
Greater Noida W ATS Destinaire ORG
Greater Noida PERSON
Residential Apartments ORG
one CARDINAL
one CARDINAL


In [141]:
for i in range(0,6):
    text=testFuncNew(df['description'][i])
    text=nlp(text)
    print(text)
    for word in text.ents:
        print(word.text,word.label_)
    

2 Bath Unfurnished 16 floor Paramount Golf Forest Ac Apartments complimented designer landscape green golf course family well deserved healthy lifestyle
2 CARDINAL
16 CARDINAL
Paramount ORG
4 Bath Semi Furnished 19 floor A life live abundant exhilarating moments always craved The stunning contemporary design redefines classic urban environment Luxury amenities distinctive lifestyle enlivened welcoming mother nature complete grand living experience Immerse enchanting luxury enjoy best life store Purvanchal Royal City
4 CARDINAL
Bath Semi ORG
19 CARDINAL
2 Bath Semi Furnished SKA Metro Ville fresh launch residential group housing project Greater Noida SKA Group SKA Metro Ville Greater Noida IGBC Gold Pre certified project aims promote green homes residents could live quality lifestyle optimal use natural resources The project covers area 20073 90 sq meter comprises four towers 1062 luxurious Apartments Designed modern style brilliant interior also basic advanced features provided enhance

In [145]:
def get_top_n_words(corpus, n=None):
    vec = CountVectorizer().fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

In [146]:
STOPWORDS=stopwords.words("english")

In [147]:
df['description_rpn']=df.description.apply(lambda x :testFuncNew(x))

In [148]:
common_words = get_top_n_words(df['description_rpn'], 100)

In [149]:
common_words

[('floor', 1969),
 ('bath', 1530),
 ('apartment', 1216),
 ('noida', 1040),
 ('multistorey', 1010),
 ('furnished', 987),
 ('greater', 891),
 ('available', 834),
 ('semi', 776),
 ('sale', 714),
 ('facing', 642),
 ('bhk', 636),
 ('unfurnished', 615),
 ('lac', 532),
 ('east', 485),
 ('located', 424),
 ('the', 407),
 ('floors', 395),
 ('total', 373),
 ('it', 326),
 ('this', 313),
 ('flat', 306),
 ('north', 286),
 ('property', 259),
 ('building', 246),
 ('area', 235),
 ('west', 228),
 ('sector', 180),
 ('room', 177),
 ('tower', 173),
 ('one', 166),
 ('covered', 165),
 ('parking', 152),
 ('spacious', 149),
 ('well', 148),
 ('10', 147),
 ('project', 139),
 ('construction', 134),
 ('kitchen', 132),
 ('space', 127),
 ('society', 127),
 ('city', 125),
 ('park', 118),
 ('villa', 113),
 ('amenities', 111),
 ('ground', 111),
 ('bedrooms', 111),
 ('pool', 110),
 ('sq', 108),
 ('living', 107),
 ('facilities', 106),
 ('residents', 105),
 ('ft', 105),
 ('12', 103),
 ('balconies', 103),
 ('good', 97),
 (

In [165]:
def get_top_n_trigram(corpus,start=1, n=None):
    vec = CountVectorizer(ngram_range=(start, 3), stop_words='english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

In [166]:
def get_top_n_ngram(corpus,start=1,end=3, n=None):
    vec = CountVectorizer(ngram_range=(start, end), stop_words='english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

In [153]:
n_gram=get_top_n_trigram(df['description_rpn'],2,50)

In [154]:
n_gram

[('multistorey apartment', 1009),
 ('greater noida', 883),
 ('semi furnished', 774),
 ('bath semi', 718),
 ('bath semi furnished', 718),
 ('available sale', 644),
 ('apartment available', 586),
 ('bath unfurnished', 584),
 ('apartment available sale', 584),
 ('multistorey apartment available', 535),
 ('bhk multistorey', 485),
 ('bhk multistorey apartment', 485),
 ('furnished floor', 484),
 ('floor bhk', 457),
 ('east facing', 446),
 ('floor bhk multistorey', 422),
 ('semi furnished floor', 376),
 ('unfurnished floor', 321),
 ('bath unfurnished floor', 318),
 ('apartment located', 279),
 ('floor multistorey', 267),
 ('floor multistorey apartment', 266),
 ('multistorey apartment located', 243),
 ('floor north', 221),
 ('north east', 208),
 ('building total', 206),
 ('north east facing', 193),
 ('floor east', 188),
 ('floor east facing', 188),
 ('bath furnished', 187),
 ('noida west', 165),
 ('furnished floor bhk', 164),
 ('greater noida west', 163),
 ('floor north east', 163),
 ('tower t

In [173]:
tri_gram=get_top_n_ngram(df['description_rpn'],3,3,25)

In [174]:
tri_gram

[('bath semi furnished', 718),
 ('apartment available sale', 584),
 ('multistorey apartment available', 535),
 ('bhk multistorey apartment', 485),
 ('floor bhk multistorey', 422),
 ('semi furnished floor', 376),
 ('bath unfurnished floor', 318),
 ('floor multistorey apartment', 266),
 ('multistorey apartment located', 243),
 ('north east facing', 193),
 ('floor east facing', 188),
 ('furnished floor bhk', 164),
 ('greater noida west', 163),
 ('floor north east', 163),
 ('floor tower total', 145),
 ('sale greater noida', 129),
 ('available sale greater', 126),
 ('available sale sector', 124),
 ('west greater noida', 112),
 ('floor building total', 111),
 ('noida west greater', 108),
 ('bath furnished floor', 108),
 ('unfurnished floor bhk', 107),
 ('apartment building total', 94),
 ('multistorey apartment building', 84)]

In [185]:
one_gram=get_top_n_ngram(df['description_rpn'],1,1,60)

In [186]:
one_gram

[('floor', 1969),
 ('bath', 1530),
 ('apartment', 1216),
 ('noida', 1040),
 ('multistorey', 1010),
 ('furnished', 987),
 ('greater', 891),
 ('available', 834),
 ('semi', 776),
 ('sale', 714),
 ('facing', 642),
 ('bhk', 636),
 ('unfurnished', 615),
 ('lac', 532),
 ('east', 485),
 ('located', 424),
 ('floors', 395),
 ('total', 373),
 ('flat', 306),
 ('north', 286),
 ('property', 259),
 ('building', 246),
 ('area', 235),
 ('west', 228),
 ('sector', 180),
 ('room', 177),
 ('tower', 173),
 ('covered', 165),
 ('parking', 152),
 ('spacious', 149),
 ('10', 147),
 ('project', 139),
 ('construction', 134),
 ('kitchen', 132),
 ('space', 127),
 ('society', 127),
 ('city', 125),
 ('park', 118),
 ('villa', 113),
 ('amenities', 111),
 ('ground', 111),
 ('bedrooms', 111),
 ('pool', 110),
 ('sq', 108),
 ('living', 107),
 ('facilities', 106),
 ('residents', 105),
 ('ft', 105),
 ('12', 103),
 ('balconies', 103),
 ('good', 97),
 ('bathrooms', 97),
 ('location', 94),
 ('constructed', 92),
 ('metro', 90),
 

In [188]:
two_gram=get_top_n_ngram(df['description_rpn'],2,2,60)

In [189]:
two_gram

[('multistorey apartment', 1009),
 ('greater noida', 883),
 ('semi furnished', 774),
 ('bath semi', 718),
 ('available sale', 644),
 ('apartment available', 586),
 ('bath unfurnished', 584),
 ('bhk multistorey', 485),
 ('furnished floor', 484),
 ('floor bhk', 457),
 ('east facing', 446),
 ('unfurnished floor', 321),
 ('apartment located', 279),
 ('floor multistorey', 267),
 ('floor north', 221),
 ('north east', 208),
 ('building total', 206),
 ('floor east', 188),
 ('bath furnished', 187),
 ('noida west', 165),
 ('tower total', 147),
 ('floor tower', 146),
 ('sale greater', 129),
 ('sale sector', 124),
 ('floor building', 114),
 ('west greater', 112),
 ('ground floor', 107),
 ('total floors', 104),
 ('apartment building', 94),
 ('facing bhk', 87),
 ('sq ft', 86),
 ('spacious multistorey', 78),
 ('newly constructed', 74),
 ('car parking', 71),
 ('sector greater', 70),
 ('12 floor', 70),
 ('noida extension', 69),
 ('floor spacious', 67),
 ('located advantageous', 67),
 ('10 floor', 65),


In [194]:
df_keyword=pd.DataFrame(one_gram,columns=['keywords','count'])

In [203]:
df_keyword.head()

,keywords,count
0,floor,1969
1,bath,1530
2,apartment,1216
3,noida,1040
4,multistorey,1010


In [196]:
df_keyword_test=pd.DataFrame(two_gram,columns=['keywords','count'])

In [204]:
df_keyword = pd.concat([df_keyword,df_keyword_test],axis=0)

In [206]:
df_keyword.head()

,keywords,count
0,floor,1969
1,bath,1530
2,apartment,1216
3,noida,1040
4,multistorey,1010


In [199]:
df_keyword_test=pd.DataFrame(tri_gram,columns=['keywords','count'])

In [209]:
df_keyword.to_csv(f'{path}keyword.csv')

In [210]:
df_keyword=pd.read_csv(f'{path}keyword.csv')

In [216]:
df_keyword.fillna('',inplace=True)

In [217]:
keywords_master=df_keyword['Keywords'].unique()

In [251]:
keywords_master

array(['floor', 'bath', 'apartment', 'multistorey', 'furnished', 'semi',
       'sale', 'facing', 'bhk', 'unfurnished', 'east', 'located',
       'floors', 'flat', 'north', 'property', 'building', 'west',
       'sector', 'room', 'covered', 'parking', 'spacious', 'construction',
       'kitchen', 'society', 'park', 'amenities', 'pool', 'balconies',
       'metro', 'constructed', 'view', 'multistorey apartment',
       'semi furnished', 'east facing', 'unfurnished floor', 'north east',
       'building total', 'floor tower', 'newly constructed',
       'car parking', 'floor apartment', 'builder floor', 'covered car',
       'covered area', 'metro station', 'swimming pool', 'west facing',
       'modular kitchen', 'north east facing', 'floor east facing', ''],
      dtype=object)

In [ ]:
def Punctuation(text): 
  
    # punctuation marks 
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
  
    # traverse the given string and if any punctuation 
    # marks occur replace it with null 
    for x in text.lower(): 
        if x in punctuations: 
            text = text.replace(x, "")
    return text

In [259]:
def generate_ngrams(text,n=3):
    text=clean_text(text)
    # split sentences into tokens
    tokens=re.split("\\s+",text)
    ngrams=[]
    for n in range(1,(n+1)):
        for i in range(len(tokens)-n+1):
            temp=[tokens[j] for j in range(i,i+n)]
            ngrams.append(" ".join(temp))
    a_set = set(keywords_master) 
    b_set = set(ngrams) 
    list_result=[]
    if (a_set & b_set): 
        list_result=(a_set & b_set) 
    else: 
        list_result=b_set
    return list_result
        

In [258]:
generate_ngrams(df.description_rpn[182],3)

{'bath',
 'construction',
 'east',
 'east facing',
 'facing',
 'flat',
 'floor',
 'floor east facing',
 'furnished',
 'metro',
 'sale',
 'semi',
 'semi furnished'}

In [260]:
df['keywords']=df.description_rpn.apply(lambda x : generate_ngrams(x))

In [261]:
df.head()

,name,id,createdDate,description,Society,url,price,priceInWord,cityName,addressLocality,...,projectName,builderName,propertyType,locality,postedBy,clean_desc,sent_tokenize,word_tokenize,description_rpn,keywords
0,2 BHK Apartment for Sale in Golfforeste AC Apa...,domcache_srp_46630105,20200211,"2 Bath,Unfurnished,16 floor Paramount Golf For...",Golfforeste AC Apartments,https://www.magicbricks.com/propertyDetails/2-...,3710000.0,37.1 Lac,Greater Noida,Zeta 1,...,Golfforeste AC Apartments,Paramount Group,Apartment,Zeta 1,Agent,2 bath unfurnished 16 floor paramount golf for...,[2 bath unfurnished 16 floor paramount golf fo...,"[2, bath, unfurnished, 16, floor, paramount, g...",2 Bath Unfurnished 16 floor Paramount Golf For...,"{floor, bath, unfurnished}"
1,4 BHK Apartment for Sale in Purvanchal Royal C...,domcache_srp_46527183,20200131,"4 Bath,Semi-Furnished,19 floor A life where yo...",Purvanchal Royal City,https://www.magicbricks.com/propertyDetails/4-...,11200000.0,1.12 Cr,Greater Noida,Chi 5,...,Purvanchal Royal City,Purvanchal Projects Pvt. Ltd.,Apartment,Chi 5,Builder,4 bath semifurnished 19 floor life live abunda...,[4 bath semifurnished 19 floor life live abund...,"[4, bath, semifurnished, 19, floor, life, live...",4 Bath Semi Furnished 19 floor A life live abu...,"{semi furnished, amenities, bath, furnished, s..."
2,"3 BHK Apartment for Sale in SKA Metro Ville, E...",domcache_srp_46365097,20200130,"2 Bath,Semi-Furnished SKA Metro Ville is the f...",SKA Metro Ville,https://www.magicbricks.com/propertyDetails/3-...,3430000.0,34.3 Lac,Greater Noida,"Eta 2, Eta",...,SKA Metro Ville,SKA Group,Apartment,"Eta 2, Eta",Builder,2 bath semifurnished ska metro ville fresh lau...,[2 bath semifurnished ska metro ville fresh la...,"[2, bath, semifurnished, ska, metro, ville, fr...",2 Bath Semi Furnished SKA Metro Ville fresh la...,"{semi furnished, metro, bath, furnished, semi,..."
3,"2 BHK Apartment for Sale in Arihant Ambar, Gre...",domcache_srp_34690341,20200207,"2 Bath,Semi-Furnished,20 floor Arihant Ambar i...",Arihant Ambar,https://www.magicbricks.com/propertyDetails/2-...,4300000.0,43 Lac,Greater Noida,Greater Noida West,...,Arihant Ambar,Arihant Group,Apartment,Greater Noida West,Builder,2 bath semifurnished 20 floor arihant ambar lu...,[2 bath semifurnished 20 floor arihant ambar l...,"[2, bath, semifurnished, 20, floor, arihant, a...",2 Bath Semi Furnished 20 floor Arihant Ambar l...,"{semi furnished, amenities, bath, furnished, s..."
4,"1 BHK Apartment for Sale in Prideville , Yamun...",domcache_srp_45300261,20200210,"1 Bath,Semi-Furnished We at Prideville underst...",Prideville,https://www.magicbricks.com/propertyDetails/1-...,3700000.0,37 Lac,Greater Noida,Yamuna Expressway,...,Prideville,Imperia Structures Ltd.,Apartment,Yamuna Expressway,Agent,1 bath semifurnished prideville understand mus...,[1 bath semifurnished prideville understand mu...,"[1, bath, semifurnished, prideville, understan...",1 Bath Semi Furnished We Prideville understand...,"{semi furnished, bath, furnished, semi, view}"


In [262]:
df.drop(columns=['clean_desc','sent_tokenize','word_tokenize'],inplace=True)

In [263]:
df.head(2).T

,0,1
name,2 BHK Apartment for Sale in Golfforeste AC Apa...,4 BHK Apartment for Sale in Purvanchal Royal C...
id,domcache_srp_46630105,domcache_srp_46527183
createdDate,20200211,20200131
description,"2 Bath,Unfurnished,16 floor Paramount Golf For...","4 Bath,Semi-Furnished,19 floor A life where yo..."
Society,Golfforeste AC Apartments,Purvanchal Royal City
url,https://www.magicbricks.com/propertyDetails/2-...,https://www.magicbricks.com/propertyDetails/4-...
price,3.71e+06,1.12e+07
priceInWord,37.1 Lac,1.12 Cr
cityName,Greater Noida,Greater Noida
addressLocality,Zeta 1,Chi 5


In [6]:
def normalize(s):
    """
    Given a text, cleans and normalizes it. Feel free to add your own stuff.
    """
    s = s.lower()
    # Replace ips
    #s = re.sub(r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}', ' _ip_ ', s)
    # Isolate punctuation
    s = re.sub(r'([\'\"\.\(\)\!\?\-\\\/\,])', r' \1 ', s)
    # Remove some special characters
    s = re.sub(r'([\;\:\|•«\n])', ' ', s)
    # Replace numbers and symbols with language
    s = s.replace('&', ' and ')
    s = s.replace('@', ' at ')
    
    return s

In [9]:
from gensim.models.fasttext import FastText
from gensim.corpora import Dictionary
from gensim.parsing.preprocessing import remove_stopwords,strip_punctuation

In [14]:
def preprocess(row):
    return strip_punctuation(remove_stopwords(row.lower()))

In [15]:
df['clean_desc']=df.description.apply(preprocess)

In [18]:
df['clean_desc'][143]

'3 bath semi furnished 3 floor north facing multistorey apartment available sale  covered area 2300 sq ft  open flat proper sun facing flat  park basketball ground facing car parking '

In [20]:
texts = []
for line in df.clean_desc:
    lowered = line.lower()
    words = re.findall(r'\ w+', lowered)
    texts.append(words)

In [21]:
texts

[[' w'],
 [' w'],
 [],
 [' w', ' w', ' w', ' w'],
 [],
 [' w'],
 [' w'],
 [],
 [' w'],
 [' w', ' w', ' w', ' w'],
 [],
 [' w', ' w'],
 [' w'],
 [' w', ' w', ' w', ' w'],
 [' w'],
 [' w', ' w', ' w', ' w'],
 [' w'],
 [],
 [' w'],
 [' w'],
 [' w'],
 [' w', ' w', ' w', ' w', ' w'],
 [],
 [' w'],
 [],
 [],
 [' w', ' w', ' w', ' w'],
 [],
 [' w'],
 [' w'],
 [],
 [' w'],
 [],
 [' w', ' w', ' w', ' w', ' w', ' w', ' w'],
 [],
 [' w', ' w'],
 [],
 [' w', ' w', ' w', ' w', ' w', ' w', ' w', ' w'],
 [' w'],
 [],
 [' w',
  ' w',
  ' w',
  ' w',
  ' w',
  ' w',
  ' w',
  ' w',
  ' w',
  ' w',
  ' w',
  ' w',
  ' w',
  ' w'],
 [' w', ' w', ' w', ' w', ' w', ' w', ' w', ' w', ' w', ' w', ' w'],
 [' w', ' w', ' w', ' w', ' w', ' w', ' w', ' w', ' w', ' w'],
 [' w'],
 [' w', ' w', ' w', ' w', ' w', ' w', ' w', ' w', ' w'],
 [],
 [' w', ' w', ' w', ' w', ' w', ' w', ' w'],
 [],
 [],
 [' w', ' w', ' w', ' w'],
 [],
 [],
 [],
 [],
 [],
 [' w', ' w', ' w', ' w', ' w', ' w', ' w'],
 [],
 [' w', ' w', ' w',